In [51]:
# Numerical Operations
import math #数学库
import numpy as np #数组与矩阵运算库

# Reading/Writing Data
import pandas as pd #数据处理
import os #operating system库 程序与系统交互
import csv #读csv文件

# For Progress Bar
from tqdm import tqdm #进度条库

# Pytorch
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

from sklearn.feature_selection import SelectKBest, f_regression

In [52]:
def same_seed(seed): 
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True #backends为后端配置 代码使得CUDACNN的随机性消失
    torch.backends.cudnn.benchmark = False #禁用性能基准测试防止调优
    np.random.seed(seed) #初始化NumPy的随机数生成种子
    torch.manual_seed(seed) #初始化Pytorch的随机数生成种子
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed) #初始化GPU中生成种子的函数

def train_valid_split(data_set, valid_ratio, seed):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * len(data_set)) #验证集大小
    train_set_size = len(data_set) - valid_set_size #训练集大小
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed)) #根据种子划分训练集与验证集
    return np.array(train_set), np.array(valid_set) #转化为np数组

def predict(test_loader, model, device): #test_loader 数据加载器 model 深度学习模型
    model.eval() # Set your model to evaluation mode.
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)                        
        with torch.no_grad():  #预测时不需要计算梯度              
            pred = model(x)                     
            preds.append(pred.detach().cpu())   #detach（）将预测值从计算图中分离 移到CPU上（节省内存；提高效率；防止梯度累计）
    preds = torch.cat(preds, dim=0).numpy()  #将预测结果拼接到NumPy数组中
    return preds

In [53]:
class AgeDataset(Dataset): #继承Dataset
    '''
    x: Features.
    y: Targets, if none, do prediction.
    '''
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

In [99]:
class My_Model(nn.Module): #
    def __init__(self, input_dim):
        super(My_Model, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(512),
            nn.Dropout(0.2),
            
            nn.Linear(512, 32),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(32),
            nn.Dropout(0.1),
            
            nn.Linear(32, 1)
            
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1)
        return x

In [62]:

# 加载数据集
data = pd.read_csv('dataset.csv')
data


,id,-,-.1,-.2,-.3,-.4,-.5,-.6,-.7,-.8,...,-.1042,-.1043,-.1044,-.1045,-.1046,-.1047,+.997,+.998,+.999,Age
0,0.0,0.001769,0.002365,0.005017,0.002751,0.004434,0.002552,0.007265,0.004367,0.004367,...,0.173913,0.173913,0.043478,0.000000,0.000000,0.000000,0.882067,0.696895,0.700976,6.0
1,1.0,0.003758,0.004354,0.002211,0.000099,0.003559,0.004742,0.007315,0.004448,0.014222,...,NaN,NaN,NaN,NaN,NaN,NaN,0.895260,0.709369,0.733696,6.0
2,2.0,0.002384,0.002018,0.000374,0.000275,0.002381,0.003482,0.001833,0.009282,0.021053,...,NaN,NaN,NaN,NaN,NaN,NaN,0.933556,0.724635,0.764805,6.0
3,3.0,0.000256,0.000256,0.009719,0.006776,0.001405,0.001022,0.001535,0.003986,0.022109,...,0.000000,0.000000,0.000000,0.064516,0.000000,0.000000,0.901789,0.643957,0.763804,15.0
4,4.0,0.003219,0.002956,0.000000,0.001611,0.000000,0.000268,0.001877,0.000000,0.024306,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.927982,0.750116,0.770227,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,117.0,0.005183,0.002898,0.003820,0.000414,0.002999,0.005839,0.005589,0.006791,0.018110,...,0.093168,0.110390,0.038462,0.064516,0.153846,0.064103,0.877246,0.689991,0.790897,19.2
118,118.0,0.000726,0.003282,0.002959,0.004270,0.004634,0.010200,0.008160,0.004632,0.012951,...,0.121951,0.050000,0.170732,0.048780,0.048780,0.000000,0.849731,0.671967,0.794059,18.5
119,119.0,0.002390,0.002679,0.004280,0.005208,0.006247,0.000297,0.000299,0.000000,0.002825,...,NaN,NaN,NaN,NaN,NaN,NaN,0.894902,0.702532,0.773831,18.5
120,120.0,0.006552,0.000160,0.000326,0.001835,0.003348,0.005917,0.000160,0.011196,0.010687,...,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.885314,0.711228,0.738259,18.0


In [63]:

def select_feat(train_data, valid_data, test_data, select_all=True):
    # Selects useful features to perform regression
    y_train, y_valid = train_data[:,-1], valid_data[:,-1] #为什么从-1开始
    raw_x_train, raw_x_valid, raw_x_test = train_data[:,:-1], valid_data[:,:-1], test_data #这两行还有区别

    if select_all:
        feat_idx = list(range(raw_x_train.shape[1])) #访问形状数组第二个元素 列数
    else:
        k = config['k'] # 所要选择的特征数量
        selector = SelectKBest(score_func=f_regression, k=k)
        result = selector.fit(train_data[:, :-1], train_data[:,-1])
        idx = np.argsort(result.scores_)[::-1]
        feat_idx = list(np.sort(idx[:k]))
        
    return raw_x_train[:,feat_idx], raw_x_valid[:,feat_idx], raw_x_test[:,feat_idx], y_train, y_valid


In [64]:
def trainer(train_loader, valid_loader, model, config, device):

    criterion = nn.MSELoss(reduction='mean') # Define your loss function, do not modify this.

    # Define your optimization algorithm. 
    # TODO: Please check https://pytorch.org/docs/stable/optim.html to get more available algorithms.
    # TODO: L2 regularization (optimizer(weight decay...) or implement by your self).
    optimizer = torch.optim.Adam(model.parameters()) 

    writer = SummaryWriter() # Writer of tensorboard.

    if not os.path.isdir('./models'):
        os.mkdir('./models') # Create directory of saving models.

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0 #训练轮数 最小损失值 当前训练步数 提前停止训练

    for epoch in range(n_epochs):
        model.train() # Set your model to train mode.
        loss_record = []

        # tqdm is a package to visualize your training progress.
        train_pbar = tqdm(train_loader, position=0, leave=True) #进度条在顶部 进度完成后不清除

        for x, y in train_pbar:
            optimizer.zero_grad()               # Set gradient to zero.
            x, y = x.to(device), y.to(device)   # Move your data to device. 
            pred = model(x)             
            loss = criterion(pred, y)
            loss.backward()                     # Compute gradient(backpropagation).
            optimizer.step()                    # Update parameters.
            step += 1
            loss_record.append(loss.detach().item())
            
            # Display current epoch number and loss on tqdm progress bar.
            train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            train_pbar.set_postfix({'loss': loss.detach().item()})

        mean_train_loss = sum(loss_record)/len(loss_record)
        writer.add_scalar('Loss/train', mean_train_loss, step) #将训练损失添加到tensorboard日志中

        model.eval() # Set your model to evaluation mode.
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())
            
        mean_valid_loss = sum(loss_record)/len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        writer.add_scalar('Loss/valid', mean_valid_loss, step) #记录验证集平均损失

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # Save your best model 状态字典
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else: 
            early_stop_count += 1 #没有提升就加一 达到阙值后停止训练

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return

In [100]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'seed': 20060512,      # Your seed number, you can pick your lucky number. :)
    'select_all': False,   # Whether to use all features.
    'valid_ratio': 0.2,   # validation_size = train_size * valid_ratio
    'n_epochs': 5000,     # Number of epochs.            
    'batch_size': 64, 
    'learning_rate': 1e-5,              
    'early_stop': 400,    # If model has not improved for this many consecutive epochs, stop training.     
    'save_path': './models/model.ckpt',  # Your model will be saved here.
    'k':2000
}

In [101]:
from sklearn.impute import SimpleImputer

# Set seed for reproducibility
same_seed(config['seed'])


train_data, test_data = pd.read_csv('./dataset.csv').values, pd.read_csv('./dataset.csv').values
imputer = SimpleImputer(strategy='mean')
train_data = imputer.fit_transform(train_data)
test_data = imputer.fit_transform(test_data)
train_data, valid_data = train_valid_split(train_data, config['valid_ratio'], config['seed'])


# Print out the data size.
print(f"""train_data size: {train_data.shape} 
valid_data size: {valid_data.shape} 
test_data size: {test_data.shape}""")

# Select features
x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data, valid_data, test_data, config['select_all'])

all_data =np.concatenate((x_train, x_valid, x_test), axis=0) #数据归一化
x_min, x_max = all_data.min(axis=0), all_data.max(axis=0)
x_train = (x_train - x_min) / (x_max - x_min)
x_valid = (x_valid - x_min) / (x_max - x_min)
x_test = (x_test - x_min) / (x_max - x_min)


# Print out the number of features.
print(f'number of features: {x_train.shape[1]}')

train_dataset, valid_dataset, test_dataset = AgeDataset(x_train, y_train), \
                                            AgeDataset(x_valid, y_valid), \
                                            AgeDataset(x_test)

# Pytorch data loader loads pytorch dataset into batches.
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True) # pin_memory:返回数据前将数据复制在CUDA固定内存中

train_data size: (98, 2050) 
valid_data size: (24, 2050) 
test_data size: (122, 2050)
number of features: 2000


In [102]:
model = My_Model(input_dim=x_train.shape[1]).to(device) # put your model and data on the same computation device.

trainer(train_loader, valid_loader, model, config, device)

Epoch [1/5000]: 100%|██████████| 2/2 [00:00<00:00, 99.42it/s, loss=216]


Epoch [1/5000]: Train loss: 261.5677, Valid loss: 214.0862
Saving model with loss 214.086...


Epoch [2/5000]: 100%|██████████| 2/2 [00:00<00:00, 162.58it/s, loss=256]


Epoch [2/5000]: Train loss: 257.2122, Valid loss: 208.8952
Saving model with loss 208.895...


Epoch [3/5000]: 100%|██████████| 2/2 [00:00<00:00, 146.02it/s, loss=252]


...

Epoch [1039/5000]: Train loss: 2.5873, Valid loss: 4.9785


Epoch [1040/5000]: 100%|██████████| 2/2 [00:00<00:00, 24.03it/s, loss=5.41]


Epoch [1040/5000]: Train loss: 4.0000, Valid loss: 4.7018


Epoch [1041/5000]: 100%|██████████| 2/2 [00:00<00:00, 227.75it/s, loss=4.25]


Epoch [1041/5000]: Train loss: 3.3664, Valid loss: 4.7209

Model is not improving, so we halt the training session.


In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/